## Comparando as performances



**Objetivo**: Compare a performance de três algoritmos diferentes de otimização (busca aleatória, busca em grade e algoritmos genéticos) para resolver o problema das caixas binárias.

**Lembrete**: nós estudamos performance de algoritmos durante lógica computacional.

**Dica**: O enunciado do objetivo não definiu o número de caixas. É esperado de um cientista que ele entenda que não adianta resolver esse problema para apenas um valor de $n$ caixas, mas sim buscar uma tendência resolvendo esse problema para alguns valores de $n$ diferentes. Fique atento pois essa dica não será mais dada a partir de agora mesmo que seja necessário usar desse bom senso científico em problemas futuros.

**Dica 2**: Lembre-se de que o único algoritmo determinístico dos três sendo estudados é o de busca em grade. Sendo assim, é esperado que um cientista entenda que situações não-determinísticas demandam o uso de estatística para quantificar um valor médio de performance e seu desvio. Novamente, fique atento pois essa dica não será mais dada a partir de agora.

**Nome do arquivo**: &ldquo;experimento GA.02 - performance caixas binarias&rdquo;



<hr>



## Importações

Neste caso, terei que importar todos os códigos utilizados nos primeiros três experimentos, visto que o nosso objetivo é comparar a performance deles.

In [1]:
from funcoes import individuo_cb, funcao_objetivo_cb
import time
import itertools
import random
from funcoes import populacao_cb as cria_populacao_cb
from funcoes import funcao_objetivo_pop_cb as funcao_objetivo_pop
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao
########## SPOILER: OTIMIZAÇÃO? ##########
import timeit
import numpy as np
from timeit import Timer

## Códigos e discussão

#### Problema 1 

In [2]:
#Constantes

NUM_CANDIDATOS = 12
NUM_GENES = 5

In [3]:
inicio = time.perf_counter()

for n in range(NUM_CANDIDATOS):
    candidato = individuo_cb(NUM_GENES)
    fobj = funcao_objetivo_cb(candidato)
    #print(candidato, fobj)
    
fim = time.perf_counter()
tempo_total = fim - inicio

print("O algoritmo com", NUM_GENES,"genes demorou", tempo_total, "segundos")

O algoritmo com 5 genes demorou 0.0002699000000001561 segundos


Vale lembrar que o objetivo deste primeiro algoritmo usado é encontrar o maior resultado da soma das casas da nossa caixa binária. Ele faz isso através de tentativa e erro, por isso, o esperado é que o tempo que ele demora para ser executado irá aumentar conforme mais casas e/ou mais possibilidades de números nessas casas (mas o problema não seria uma caixa binária nesse caso).

#### OBS.:
- A ideia de utilizar apenas o perf counter me pareceu muito simples, portanto, decidi pesquisar sobre alguns métodos de como otimizar a sua utilização. Percebi que cada contagem resultava em um número com uma diferença significativa. Assim, decidi calcular a média de _n_ runs de determinado código, ainda utilizando o perf_conuter. Tentei achar alguma biblioteca que fizesse isso, mas não pude achar nenhuma. Junto à implementação mais "complexa" do perf_counter, também pude aprender a utilizar o tipo de string f"".
- Além disso, também tentei implementar outros métodos da biblioteca time, tais como o process_time, o qual conta o tempo gasto pelo processador para rodar o código. Mas, por algum erro, ou pouco uso do processador, o resultado retornado sempre era de 0.0000 segundos. 

In [4]:
EXECS = 100
tempo_total = 0

for i in range(EXECS):
    START = time.perf_counter()

    for n in range(NUM_CANDIDATOS):
        candidato = individuo_cb(NUM_GENES)
        fobj = funcao_objetivo_cb(candidato)
        #print(candidato, fobj) Caso optar por printar, favor tomar cuidado com o número de linhas.

    END = time.perf_counter()
    tempo_total += (END - START)
    
MEDIA = (tempo_total / EXECS)
print(f"Tempo médio: {MEDIA:.6f} segundos")

Tempo médio: 0.000075 segundos


#### Problema 2

In [5]:
inicio = time.perf_counter()

for gene1 in [0, 1]:
    for gene2 in [0, 1]:
        for gene3 in [0, 1]:
            for gene4 in [0, 1]:
                individuo = [gene1, gene2, gene3, gene4]
                fobj = funcao_objetivo_cb(individuo)
                #print(individuo, fobj)
                
#####                
for individuo in itertools.product([0,1], [0,1], [0,1], [0,1, 2]):
    fobj = funcao_objetivo_cb(individuo)
    #print(individuo, fobj)
    
#####
for individuo in itertools.product([0,1], repeat=6):
    fobj = funcao_objetivo_cb(individuo)
    #print(individuo, fobj)
    
fim = time.perf_counter()

print("Os algoritmos com 4 e 6 genes demoraram, no total,", fim - inicio, "segundos")

Os algoritmos com 4 e 6 genes demoraram, no total, 0.00023580000000045231 segundos


In [6]:
EXECS = 100
tempo_total = 0

for i in range(EXECS):
    START = time.perf_counter()

    for gene1 in [0, 1]:
        for gene2 in [0, 1]:
            for gene3 in [0, 1]:
                for gene4 in [0, 1]:
                    individuo = [gene1, gene2, gene3, gene4]
                    fobj = funcao_objetivo_cb(individuo)
                    #print(individuo, fobj)
                
#####                
    for individuo in itertools.product([0,1], [0,1], [0,1], [0,1, 2]):
        fobj = funcao_objetivo_cb(individuo)
        #print(individuo, fobj)
    
#####
    for individuo in itertools.product([0,1], repeat=6):
        fobj = funcao_objetivo_cb(individuo)
        #print(individuo, fobj)
    
END = time.perf_counter()
tempo_total += (END - START)
    
MEDIA = (tempo_total / EXECS)
print(f"Tempo médio: {MEDIA:.7f} segundos")

Tempo médio: 0.0000022 segundos


#### Problema 3

In [7]:
# constantes

tamanho_pop = 5
numero_genes = 4
numero_geracoes = 5
chance_cruzamento = 0.5
chance_mutacao = 0.05

In [8]:
inicio = time.perf_counter()

populacao = cria_populacao_cb(tamanho_pop,numero_genes)

print("População inicial:")
print(populacao)

for n in range(numero_geracoes):
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao,fitness)
    pais = populacao[0::2]
    maes = populacao[1::2]
    contador = 0
    for pai,mae in zip(pais,maes):
        if random.random() < chance_cruzamento:
            filho1,filho2 = funcao_cruzamento(pai,mae)
            populacao[contador] = filho1
            populacao[contador+1] = filho2
            
        contador = contador + 2
        
    for individuo in range(len(populacao)):
        if random.random() <= chance_mutacao:
            individuo = populacao[n]
            
            populacao[n] = funcao_mutacao(individuo)
        
print()
print("População final:")
print(populacao)

fim = time.perf_counter()

print("O algoritmo com 4 genes demorou", fim - inicio, "segundos")

População inicial:
[[1, 1, 1, 1], [1, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0]]


IndexError: list assignment index out of range

In [10]:
inicio = time.perf_counter()

populacao = cria_populacao_cb(tamanho_pop,numero_genes)

print("População inicial:")
print(populacao)

for n in range(numero_geracoes):
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao,fitness)
    pais = populacao[0::2]
    maes = populacao[1::2]
    contador = 0
    for pai,mae in zip(pais,maes):
        if random.random() < chance_cruzamento:
            filho1,filho2 = funcao_cruzamento(pai,mae)
            populacao[contador] = filho1
            populacao[contador+1] = filho2
            
        contador = contador + 2
        
    for individuo in range(len(populacao)):
        if random.random() <= chance_mutacao:
            individuo = populacao[n]
            
            populacao[n] = funcao_mutacao(individuo)
        
print()
print("População final:")
print(populacao)

fim = time.perf_counter()

print("O algoritmo com 4 genes demorou", fim - inicio, "segundos.")

População inicial:
[[1, 0, 0, 0], [0, 0, 0, 0], [0, 1, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0]]

População final:
[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]
O algoritmo com 4 genes demorou 0.0008102000000000942 segundos


In [ ]:
EXECS = 1
tempo_total = 0

for i in range(EXECS):
    START = time.perf_counter()

    populacao = cria_populacao_cb(tamanho_pop,numero_genes)

    print("População inicial:")
    print(populacao)

    for n in range(numero_geracoes):
        fitness = funcao_objetivo_pop(populacao)
        populacao = funcao_selecao(populacao,fitness)
        pais = populacao[0::2]
        maes = populacao[1::2]
        contador = 0
        for pai,mae in zip(pais,maes):
            if random.random() < chance_cruzamento:
                filho1,filho2 = funcao_cruzamento(pai,mae)
                populacao[contador] = filho1
                populacao[contador+1] = filho2
            
            contador = contador + 2
        
        for individuo in range(len(populacao)):
            if random.random() <= chance_mutacao:
                individuo = populacao[n]
            
                populacao[n] = funcao_mutacao(individuo)
        
    print()
    print("População final:")
    print(populacao)

END = time.perf_counter()
tempo_total += (END - START)
    
MEDIA = (tempo_total / EXECS)
print(f"Tempo médio: {MEDIA:.7f} segundos")

#### OBS.:
Neste caso, não pude entender o que aconteceu, visto que em determinados momentos a implementação do perf_counter não funcionou neste código. Ao executá-lo mais de uma vez (EXECS > 1), era retornado o erro "list assignment index out of range". Isso também ocorreu mesmo sem a implementação do contador aprimorado. Além desse problema, mesmo executando o código apenas uma vez, sempre há uma grande discrepância entre os valores do tempo contado. Outro erro que não consigo desvendar.

## Conclusão

Dessa forma, podemos concluir que, o algoritmo que requer menos tempo para ser executado e retornar os resultados esperados é o segundo - de _busca em grade_. Pelo que pude entender, isso ocorre devido a ele ser um _algoritmo determinístico_, assim como citado pelo professor no enunciado. 
Um algoritmo determinístico consiste na resolução de um problema no qual cada passo será **determinado** pela sua entrada e pelos passos anteriores. Logo, o resultado final é o mesmo dependendo dos inputs utilizados, o que o torna previsível e de fácil repetição. Por isso, faz sentido que o seu tempo de execução seja menor em comparação aos outros códigos.